## Some helpful code is provided bellow. It can serve to load data and organice functions.

In [1]:
#!pip install yfinance
import yfinance as yf
precios = yf.Ticker('XLI').history(period="10y", interval='1wk')

In [2]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
from sklearn.svm import SVC

In [3]:
precios = precios.Close

In [4]:
precios.head(10)

Date
2014-03-31    43.539558
2014-04-07    42.180508
2014-04-14    43.713585
2014-04-21    43.514698
2014-04-28    43.771591
2014-05-05    43.929043
2014-05-12    43.995342
2014-05-19    44.376534
2014-05-26    44.790886
2014-06-02    45.835045
Name: Close, dtype: float64

In [5]:
retornos = np.log(precios).diff()[1:].dropna()
mat_retornos = retornos.values

In [34]:
np.set_printoptions(suppress=True)

In [122]:
def preparar(per = 20, umbral = 2):        
        
        # ventana de tiempo - Columnas
        muestras = len(mat_retornos) - per

        variables = np.zeros((1,per))
        l = len(mat_retornos)

        for x in range(muestras):
            variables = np.concatenate((variables, mat_retornos[l-per-x: l-x].reshape((1,per))))

        variables = variables[1:]

        X = variables[:,:-1]*100
        Y = variables[:,-1]*100
        
        neutral = [not x for x in ((Y>umbral)|(Y<-umbral))]
        
        Y[Y<umbral] = 0
        Y[Y>umbral] = 1
        
        #Y[Y<-umbral] = -1
        #Y[neutral] = 0
        
        i_entrenamiento = int(len(variables)*0.7) #la fila hasta donde va el entrenamiento y donde comienza el conjunto de prueba
        
        x_train = X[:i_entrenamiento, :]
        x_test =  X[i_entrenamiento:, :]
        y_train = Y[:i_entrenamiento]
        y_test =  Y[i_entrenamiento:]
        
        return x_train, x_test, y_train, y_train

In [133]:
x_train, x_test, y_train, y_train = preparar(per = 20, umbral = 1)

In [134]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense


In [140]:
# Crear el modelo
model = Sequential()

# Agregar capa oculta
model.add(Dense(5, input_dim=19, activation='sigmoid'))
model.add(Dense(3, input_dim=5, activation='sigmoid'))
#model.add(Dense(3, input_dim=3, activation='tanh'))

In [141]:
# Agregar capa de salida
model.add(Dense(1, activation='sigmoid'))

In [142]:
# Compilar el modelo
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

# Entrenar el modelo
model.fit(x_train, y_train, epochs=300, verbose=1)

Epoch 1/300
11/11 [==============================] - 1s 2ms/step - loss: 0.3000 - accuracy: 0.3862
Epoch 2/300
11/11 [==============================] - 0s 3ms/step - loss: 0.2953 - accuracy: 0.3862
Epoch 3/300
11/11 [==============================] - 0s 2ms/step - loss: 0.2913 - accuracy: 0.3862
Epoch 4/300
11/11 [==============================] - 0s 3ms/step - loss: 0.2870 - accuracy: 0.3862
Epoch 5/300
11/11 [==============================] - 0s 3ms/step - loss: 0.2833 - accuracy: 0.3862
Epoch 6/300
11/11 [==============================] - 0s 3ms/step - loss: 0.2799 - accuracy: 0.3862
Epoch 7/300
11/11 [==============================] - 0s 3ms/step - loss: 0.2764 - accuracy: 0.3862
Epoch 8/300
11/11 [==============================] - 0s 3ms/step - loss: 0.2732 - accuracy: 0.3862
Epoch 9/300
11/11 [==============================] - 0s 4ms/step - loss: 0.2703 - accuracy: 0.3862
Epoch 10/300
11/11 [==============================] - 0s 3ms/step - loss: 0.2675 - accuracy: 0.3862
Epoch 11/

In [146]:
# Evaluar el modelo
loss, accuracy = model.evaluate(x_train, y_train)
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Predicción
predicted_output = model.predict(x_test)
print("Predicción final:")
print(predicted_output[predicted_output>0.5])

11/11 [==============================] - 0s 3ms/step - loss: 0.1839 - accuracy: 0.7515
Loss: 0.1838931441307068, Accuracy: 0.7514970302581787
5/5 [==============================] - 0s 6ms/step
Predicción final:
[0.54020154 0.6079619  0.61532533 0.59527767 0.7186698  0.5919774
 0.52744776 0.65321577 0.6115153  0.5896607  0.5077827  0.5781061
 0.5365991  0.75244063 0.5283889  0.5500526  0.5623372  0.6194429
 0.57588553 0.50420505 0.7279315  0.60342    0.7637232  0.5785586
 0.7626123  0.561837   0.567982   0.7717399  0.6174241  0.5824497
 0.6347101  0.6586748  0.5481546  0.58862567 0.77225214 0.73189354]
